# GUV Phase Partitioning Analysis Pipeline

## Publication Reference
This computational pipeline is the official implementation for the analysis presented in:
> **Hierarchy of Hydrophobic and Electrostatic Interactions in DNA–Membrane Phase Selectivity** 
> *Siu Ho Wong, Yameng Lou, Yuduo Chen, Diana Morzy, and Maartje M.C. Bastings* 
> **ACS Applied Materials & Interfaces 2025** 17 (46), 63871-63881  
> [DOI: 10.1021/acsami.5c13271](https://doi.org/10.1021/acsami.5c13271)

## Overview
This notebook quantifies the spatial partitioning of fluorescent markers in giant unilamellar vesicles (GUVs) exhibiting liquid-disordered ($L_d$) and liquid-ordered ($L_o$) phase separation. 

### Pipeline Stages:
1. **Detection:** Gaussian Smoothing and Hough Circle Transform for vesicle localization.
2. **Masking:** Ring mask generation refined by DNA-channel marker distribution.
3. **Segmentation:** Angular Gap Analysis to differentiate $L_d$ and $L_o$ phases.
4. **Quantification:** Mean fluorescence intensity sampling along membrane contours.

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import binary_dilation, disk, closing, remove_small_objects, skeletonize
from scipy.ndimage import label
from scipy.optimize import least_squares

# =============================================================================
# 1. DEFINE GENERIC PATHS
# =============================================================================
base_dir = os.getcwd()
image_folder = os.path.join(base_dir, "data", "raw")
newpath = os.path.join(base_dir, "data", "processed")
file_output = os.path.join(newpath, "Analysis_Results.csv")

if not os.path.exists(newpath):
    os.makedirs(newpath)
    print(f"Created output directory at: {newpath}")

image_counter = 0

# =============================================================================
# 2. ADJUSTABLE PARAMETERS
# =============================================================================
enlargement_factor = 1.2
inner_percentage = 0.7
threshold_method = 'otsu'
fixed_threshold = 100

def compute_threshold(pixels, method, fixed_threshold=None):
    if len(pixels) == 0: return 0
    if method == 'otsu': return threshold_otsu(pixels)
    elif method == 'mean': return np.mean(pixels)
    elif method == 'median': return np.median(pixels)
    elif method == 'fixed': return fixed_threshold
    elif method == 'mean_plus_std': return np.mean(pixels) + np.std(pixels)
    return threshold_otsu(pixels)

def fit_circle(points):
    def calc_R(xc, yc): return np.sqrt((points[:, 0] - xc)**2 + (points[:, 1] - yc)**2)
    def f_2(c): return calc_R(*c) - calc_R(*c).mean()
    center_estimate = np.mean(points, axis=0)
    result = least_squares(f_2, center_estimate)
    xc, yc = result.x
    return xc, yc, calc_R(xc, yc).mean()

# =============================================================================
# 3. MAIN LOOP
# =============================================================================
if not os.path.exists(image_folder):
    print(f"Error: Folder '{image_folder}' not found. Path searched: {image_folder}")
else:
    print(f"Starting analysis on images in: {image_folder}")
    for library in os.listdir(image_folder):
        lib_path = os.path.join(image_folder, library)
        if os.path.isdir(lib_path) and "-d84" in library:
            for image_filename in os.listdir(lib_path):
                if "ome.tif" in image_filename:
                    image_counter += 1
                    image_multi = imread(os.path.join(lib_path, image_filename))
                    # Extracting channels: GUV (0) and DNA Marker (1)
                    image = image_multi[0].copy()
                    DNA_image = image_multi[1].copy()
                    
                    blurred_image = gaussian(image, sigma=1.5, preserve_range=True)
                    edges = cv2.Canny(np.uint8(blurred_image), 40, 120)
                    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1, minDist=80,
                                               param1=50, param2=30, minRadius=10, maxRadius=100)
                    
                    if circles is None: continue
                    circles = circles[0]
                    
                    # DNA-based mask refinement
                    blurred_DNA_image = gaussian(DNA_image, sigma=2, preserve_range=True)
                    thresh_DNA = threshold_otsu(blurred_DNA_image)
                    mask4 = blurred_DNA_image > thresh_DNA
                    
                    df_final = pd.DataFrame()

                    for i, (x, y, r) in enumerate(circles, start=1):
                        enlarged_radius = r * enlargement_factor
                        inner_radius = enlarged_radius * inner_percentage
                        yy, xx = np.ogrid[:image.shape[0], :image.shape[1]]
                        ring_mask = ((yy - y)**2 + (xx - x)**2 <= enlarged_radius**2) & \
                                    ((yy - y)**2 + (xx - x)**2 > inner_radius**2)
                        
                        # Circle fitting refinement
                        valid_pixels_mask = mask4 & ring_mask
                        valid_points = np.column_stack(np.where(valid_pixels_mask))[:,[1,0]]
                        
                        if len(valid_points) > 3:
                            xc_fit, yc_fit, r_fit = fit_circle(valid_points)
                            final_ring_mask = ((yy - yc_fit)**2 + (xx - xc_fit)**2 <= r_fit**2) & \
                                              ((yy - yc_fit)**2 + (xx - xc_fit)**2 > (r_fit - 1)**2)
                        else:
                            final_ring_mask = ring_mask

                        # Ld/Lo Phase Segmentation
                        pixels_in_ring = image[ring_mask]
                        if len(pixels_in_ring) > 0:
                            threshold = compute_threshold(pixels_in_ring, threshold_method, fixed_threshold)
                            bright_pixels_mask = pixels_in_ring > threshold
                            
                            bright_mask_full = np.zeros_like(image, dtype=bool)
                            ring_yy, ring_xx = np.where(ring_mask)
                            bright_mask_full[ring_yy, ring_xx] = bright_pixels_mask
                            
                            # Connectivity labeling and small object removal
                            labeled_bright, num_labels = label(bright_mask_full, structure=np.ones((3,3)))
                            if num_labels > 0:
                                filtered_bright = remove_small_objects(labeled_bright, min_size=50)
                                valid_pixels = filtered_bright[ring_yy, ring_xx] > 0
                            else: valid_pixels = np.zeros_like(bright_pixels_mask, dtype=bool)
                        else: valid_pixels = np.zeros_like(ring_mask, dtype=bool)

                        # Angular Gap Analysis for Ld/Lo Arc differentiation
                        angles = np.arctan2(ring_yy - y, ring_xx - x)
                        angles = (angles + 2 * np.pi) % (2 * np.pi)
                        valid_angles = angles[valid_pixels]
                        
                        if len(valid_angles) > 0:
                            sorted_angles = np.sort(valid_angles)
                            diffs = np.diff(sorted_angles)
                            wrap_diff = (sorted_angles[0] - sorted_angles[-1] + 2 * np.pi) % (2 * np.pi)
                            max_gap = np.max(diffs) if len(diffs) > 0 else 0
                            if wrap_diff > max_gap:
                                start_angle, end_angle = sorted_angles[0], sorted_angles[-1] + 2 * np.pi
                            else:
                                max_gap_idx = np.argmax(diffs)
                                start_angle, end_angle = sorted_angles[max_gap_idx + 1], sorted_angles[max_gap_idx] + 2 * np.pi
                            
                            # Final Ld/Lo Mask generation
                            in_ld = (angles >= start_angle % (2*np.pi)) & (angles <= end_angle % (2*np.pi)) if (start_angle % (2*np.pi)) < (end_angle % (2*np.pi)) else (angles >= start_angle % (2*np.pi)) | (angles <= end_angle % (2*np.pi))
                            Ld_arc_mask = np.zeros_like(image, dtype=bool)
                            Ld_arc_mask[ring_yy[in_ld], ring_xx[in_ld]] = True
                        else: Ld_arc_mask = np.zeros_like(image, dtype=bool)

                        # Skeletonization for intensity sampling
                        mask2_i = closing(final_ring_mask & Ld_arc_mask, disk(3))
                        mask3_i = closing(final_ring_mask & ~Ld_arc_mask, disk(3))
                        mask2_skel = binary_dilation(skeletonize(mask2_i), disk(1))
                        mask3_skel = binary_dilation(skeletonize(mask3_i), disk(1))
                        
                        # Recording results to DataFrame
                        row = pd.DataFrame({
                            'vesicle_label': i, 'library': library, 
                            'mean_DNA(Lo)': np.mean(DNA_image[mask3_skel]) if np.any(mask3_skel) else np.nan,
                            'mean_DNA(Ld)': np.mean(DNA_image[mask2_skel]) if np.any(mask2_skel) else np.nan
                        }, index=[0])
                        df_final = pd.concat([df_final, row], ignore_index=True)
                    
                    # Saving processed data to CSV
                    df_final.to_csv(file_output, mode='a' if image_counter > 1 else 'w', index=False, header=(image_counter==1))
    print(f"--- Analysis Finished. Final CSV saved to: {file_output} ---")